<a href="https://colab.research.google.com/github/lassewardenaer/ICS635-FinalProject/blob/main/ICS635FP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown 1HCVzDLZfh4Bg2-zZl9NG2IsO4OhsK3dn
!gdown 1BQoNrroTwOgH5oq1-uOpcXH4DnvBK0Pj

Downloading...
From: https://drive.google.com/uc?id=1HCVzDLZfh4Bg2-zZl9NG2IsO4OhsK3dn
To: /content/reviewdata.csv
100% 11.8M/11.8M [00:00<00:00, 98.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BQoNrroTwOgH5oq1-uOpcXH4DnvBK0Pj
To: /content/preprocessing.py
100% 366/366 [00:00<00:00, 633kB/s]


In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np

data = pd.read_csv('./reviewdata.csv')

data = data.drop(['Unique_ID', 'Category', 'Review_Header', 'Rating'], axis=1)
data = data.replace({'Own_Rating':{'Positive':1}})
data = data.replace({'Own_Rating':{'Neutral':0}})
data = data.replace({'Own_Rating':{'Negative':-1}})
data.to_csv('preprocessed_reviewdata.csv', index=False)

data.head()

,Review_text,Own_Rating
0,I liked it,1
1,I bought the phone on Amazon and been using my...,1
2,"Awesome book at reasonable price, must buy ......",1
3,good,1
4,"The book is fine,not bad,contains nice concept...",0


In [3]:
data = pd.read_csv('./preprocessed_reviewdata.csv')
print(data.shape)

data.head()

(60889, 2)


,Review_text,Own_Rating
0,I liked it,1
1,I bought the phone on Amazon and been using my...,1
2,"Awesome book at reasonable price, must buy ......",1
3,good,1
4,"The book is fine,not bad,contains nice concept...",0


In [4]:
# https://www.youtube.com/watch?v=hOCDJyZ6quA&t=527s&ab_channel=codebasics

# Check how balanced the training data is
data.value_counts(['Own_Rating']) 

Own_Rating
 1            47436
-1             9087
 0             4366
dtype: int64

In [5]:
# Balance data
positive_samples = data[data['Own_Rating']==1]
negative_samples = data[data['Own_Rating']==-1]
neutral_samples = data[data['Own_Rating']==0]

positive_samples = positive_samples.sample(neutral_samples.shape[0])
negative_samples = negative_samples.sample(neutral_samples.shape[0])

downsampled_data = pd.concat([positive_samples, negative_samples, neutral_samples])
downsampled_data.value_counts(['Own_Rating']) 

Own_Rating
-1            4366
 0            4366
 1            4366
dtype: int64

In [6]:
print(downsampled_data.shape)

(13098, 2)


In [7]:
# https://www.youtube.com/watch?v=kxeyoyrf2cM&t=1066s&ab_channel=PythonEngineer
# https://www.tensorflow.org/text/guide/word_embeddings

import re
import string

def remove_none_important_data(text):
  translator = str.maketrans("", "", string.punctuation)
  return text.translate(translator)

downsampled_data = downsampled_data.fillna({'Review_text':' '})
downsampled_data['Review_text'] = downsampled_data['Review_text'].astype(str)
downsampled_data['Review_text'] = downsampled_data.Review_text.map(remove_none_important_data)
downsampled_data.head()

,Review_text,Own_Rating
22159,One of the best self help book,1
34129,5 star for Huawei and only 3 stars for amazon ...,1
3141,Its very good,1
44506,Mst hai,1
54139,Installations side bad,1


In [8]:
!pip install nltk

In [9]:
# Remove stop words - words that give little information
# https://pythonspot.com/nltk-stop-words/

import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_specified = set(stopwords.words('english'))

def clean_stop_words(text):
  words = [word.lower() for word in text.split() if word.lower() not in stop_specified]
  return " ".join(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
downsampled_data['Review_text'] = downsampled_data.Review_text.map(clean_stop_words)
downsampled_data.head()

,Review_text,Own_Rating
22159,one best self help book,1
34129,5 star huawei 3 stars amazon including earphones,1
3141,good,1
44506,mst hai,1
54139,installations side bad,1


In [11]:
# Find number of unique words
# https://www.hackerrank.com/challenges/collections-counter/problem

from collections import Counter

def count_words(text_samples):
  counter = Counter()
  for sample in text_samples.values:
    for word in sample.split():
      counter[word] += 1
  return counter

counter = count_words(downsampled_data['Review_text'])

In [12]:
counter.most_common(5)

[('good', 5700),
 ('phone', 4655),
 ('product', 2838),
 ('quality', 2809),
 ('camera', 2806)]

In [13]:
num_unique_words = len(counter)
print(num_unique_words)

16777


In [14]:
# Split into training, validataion and test samples 

num_train = int(downsampled_data.shape[0] * 0.75)
num_val = int(downsampled_data.shape[0] * 0.15)
num_test = int(downsampled_data.shape[0] * 0.10)

X = downsampled_data.Review_text.to_numpy()
y = downsampled_data.Own_Rating.to_numpy()

from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=0)

X_train = X[0:num_train]
X_test = X[num_train:(num_train + num_test)]
X_val = X[(num_train + num_test):]

from tensorflow.python.keras.utils import np_utils

y = np_utils.to_categorical(y, 3)

y_train = y[0:num_train]
y_test = y[num_train:(num_train + num_test)]
y_val = y[(num_train + num_test):]

print(X_train)

['everything phone good main issue battery backupit meets requirement heavy users'
 'satisfied product value money' 'great' ...
 'value money launched 8990 1 week prize dropped 7990' 'nice'
 '1st oneplus phoneand meet expectations']


In [15]:
# Use Tokenizer to transform text integer sequences

from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(X_train)
X = np.concatenate([X_train, X_test, X_val], axis=0)
X = tokenizer.texts_to_sequences(X)
X_train = X[0:num_train]
X_test = X[num_train:(num_train + num_test)]
X_val = X[(num_train + num_test):]

In [16]:
# To make all entries the same length we do padding 
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LEN = 40

X_train_padded = pad_sequences(X_train, maxlen=MAX_LEN, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test, maxlen=MAX_LEN, padding='post', truncating='post')
X_val_padded = pad_sequences(X_val, maxlen=MAX_LEN, padding='post', truncating='post')

In [17]:
# https://datascience.stackexchange.com/questions/48246/how-to-compute-f1-in-tensorflow

import keras.backend as K

def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
# https://www.tensorflow.org/text/tutorials/text_classification_rnn
import keras 

lr_list = [4e-4]
num_nodes_in_dense = [20, 40, 80]
embedding_output = [10]
lstm_list = [8]
results = []
indces = []

EPOCHS = 6
BATCH_SIZE = 10

for lr in lr_list:
  for nodes in num_nodes_in_dense:
    for emb in embedding_output:
      for lstm in lstm_list:
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Embedding(num_unique_words, emb, input_length=MAX_LEN))
        model.add(tf.keras.layers.LSTM(lstm))
        model.add(tf.keras.layers.Dense(nodes, activation='relu'))
        model.add(tf.keras.layers.Dense(3, activation='sigmoid'))

        model.compile(
          loss='categorical_crossentropy', 
          optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
          metrics=[f1_metric, 'accuracy']
          )

        result = model.fit(
          X_train_padded, 
          y_train,
          shuffle=True,
          epochs=EPOCHS,
          validation_data=(X_val_padded, y_val),
          batch_size=BATCH_SIZE
          )
        results.append(result)
        print('Parameters: ', [lr, nodes, emb, lstm])
        indces.append([lr, nodes, emb, lstm])

In [26]:
i = 0
f1 = 0
max_i = 0
for result in results:
  if result.history['val_f1_metric'][-1] > f1:
    f1 = result.history['val_f1_metric'][-1]
    max_i = i
  i += 1

In [ ]:
print('f1: ', f1)
print('Index parameters: ', max_i)
print('Parameters: ', indces[max_i])

In [ ]:
[lr, nodes, emb, lstm] = indces[max_i]

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(num_unique_words, emb, input_length=MAX_LEN))
model.add(tf.keras.layers.LSTM(lstm))
model.add(tf.keras.layers.Dense(nodes, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='sigmoid'))

model.compile(
  loss='categorical_crossentropy', 
  optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
  metrics=[f1_metric, 'accuracy']
  )

result = model.fit(
  X_train_padded, 
  y_train,
  shuffle=True,
  epochs=EPOCHS,
  validation_data=(X_val_padded, y_val),
  batch_size=BATCH_SIZE
  )

In [23]:
model.evaluate(X_test_padded, y_test)

41/41 [==============================] - 0s 4ms/step - loss: 0.9469 - f1_metric: 0.6371 - accuracy: 0.6142


[0.9469155669212341, 0.6371011137962341, 0.6142092943191528]